In [1]:
#default_exp anime.dataloader

In [2]:
#export
import pandas as pd
from pathlib import Path
import json
from functools import partial
from PIL import Image
import numpy as np
from itertools import chain
from local.torch_basics import *
from local.test import *
from local.layers import *
from local.data.all import *
from local.data.block import *
from local.optimizer import *
from local.learner import *
from local.metrics import *
from local.callback.all import *
from local.vision.all import *

In [3]:
path=Path("/home/fast/.fastai/data/danbooru2018")

In [4]:
meta_path=path/'metadata/2018000000000016'

In [5]:
portraits=pd.read_pickle(path/'portraits')

In [6]:
selfie_path=Path('/home/fast/.fastai/data/Selfie-dataset')
self_txt= selfie_path/'selfie_dataset.txt'
self_txt.exists()

True

In [7]:
attrs='image_name popularity partial_faces is_female baby child teenager youth middle_age senior white black asian oval_face round_face heart_face smiling mouth_open frowning wearing_glasses wearing_sunglasses wearing_lipstick tongue_out duck_face black_hair blond_hair brown_hair red_hair curly_hair straight_hair braid_hair showing_cellphone using_earphone using_mirror braces wearing_hat harsh_lighting dim_lighting'.split()

In [8]:
selfies=pd.read_pickle(selfie_path/'selfies')

## Tag comparison

In [9]:
def getSelfiePath(df,dir,i):
    dft=df.iloc[[i]]
    path=dir/((dft['image_name']).values[0]+'.jpg')
    return (dft.values,path)
def getImagePath(df,dir,i):
    dft=df.iloc[[i]]
    path=dir/'512px'/(dft['directory']+'/'+dft['file']).values[0]
    return (dft.tags.values[0],path)

In [10]:
def txt_to_path(img,dir=dir,ext='.jpg'):
    return dir/(img+ext)
selfie_imgs=L(list(selfies['image_name'].apply(partial(txt_to_path,dir=selfie_path/'images'))))

In [11]:
f=partial(txt_to_path,dir=path/'512px',ext='')
locs=portraits['directory']+'/'+portraits['file']
anime_imgs=L(list(locs.apply(f)))

In [12]:
len(anime_imgs),len(selfie_imgs)

(436, 46836)

In [13]:
tfms = [PILImage.create]
ds_img_tfms = [ToTensor()]

In [14]:
class CycleImage(Tuple):
    def show(self, ctx=None, **kwargs): 
        img1,img2 = self
        return show_image(torch.cat([img1,img2], dim=2),  ctx=ctx)

In [15]:
class CyclePair(Transform):
    def __init__(self,bItems):
        self.small = bItems #(aItems,bItems) if len(aItems)>len(bItems) else (bItems,aItems)
        self.assoc = self
        
    def encodes(self,i):
        "x: tuple of `i`th image and a random image from same or different class; y: True if same class"
        return CycleImage(i, random.choice(self.small)) #CycleImage(self.large[i], random.choice(self.small))

In [16]:
def resized_image(fn:Path, sz=128):
    x = Image.open(fn).resize((sz,sz))
    # Convert image to tensor for modeling
    x = array(x)
    if x.ndim==3:
        return tensor(x).permute(2,0,1).float()/255.
    elif x.ndim==2:
        return tensor(x)[None].repeat(3,1,1).float()/255.
OpenAndResize = TupleTransform(resized_image)

In [17]:
tm=[CyclePair(anime_imgs), OpenAndResize]+[ToTensor(),ByteToFloatTensor()]

In [18]:
class ToTensor(TupleTransform):
    "Convert item to appropriate tensor class"
    order = 15
@ToTensor
def encodes(self, o:CycleImage): return TensorImage(image2byte(o))

In [19]:
after_batch=[Cuda()]#Cuda(),#+aug_transforms(do_flip=True, flip_vert=True, max_rotate=10., max_zoom=1.1, max_lighting=0.2,
                   #max_warp=0.2, p_affine=0.75, p_lighting=0.75, xtra_tfms=None,
                   #size=None, mode='bilinear', pad_mode=PadMode.Reflection)
tfm=Pipeline(tm, as_item=False) #+after_batch

In [20]:
tfm2=Pipeline([OpenAndResize]+[ToTensor(as_item=False),ByteToFloatTensor()],as_item=False) #+after_batch

In [21]:
class NoopTensor(Transform):
    def encodes(self, x): return torch.FloatTensor([0])

In [22]:
tfm2=Pipeline([NoopTensor],as_item=False)

In [23]:
tfmDs=DataSource(selfie_imgs,[tfm,tfm2])

In [24]:
dl=TfmdDL(tfmDs,bs=2, shuffle=False, num_workers=8,after_batch=after_batch)

In [25]:
db=DataBunch(dl)

In [26]:
class NoValidCallback(Callback) :
    def after_train(self):
        raise CancelEpochException

In [27]:
#networks
class ResnetGenerator(nn.Module):
    def __init__(self, input_nc, output_nc, ngf=64, n_blocks=6, img_size=256, light=False):
        assert(n_blocks >= 0)
        super(ResnetGenerator, self).__init__()
        self.input_nc = input_nc
        self.output_nc = output_nc
        self.ngf = ngf
        self.n_blocks = n_blocks
        self.img_size = img_size
        self.light = light

        DownBlock = []
        DownBlock += [nn.ReflectionPad2d(3),
                      nn.Conv2d(input_nc, ngf, kernel_size=7, stride=1, padding=0, bias=False),
                      nn.InstanceNorm2d(ngf),
                      nn.ReLU(True)]

        # Down-Sampling
        n_downsampling = 2
        for i in range(n_downsampling):
            mult = 2**i
            DownBlock += [nn.ReflectionPad2d(1),
                          nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3, stride=2, padding=0, bias=False),
                          nn.InstanceNorm2d(ngf * mult * 2),
                          nn.ReLU(True)]

        # Down-Sampling Bottleneck
        mult = 2**n_downsampling
        for i in range(n_blocks):
            DownBlock += [ResnetBlock(ngf * mult, use_bias=False)]

        # Class Activation Map
        self.gap_fc = nn.Linear(ngf * mult, 1, bias=False)
        self.gmp_fc = nn.Linear(ngf * mult, 1, bias=False)
        self.conv1x1 = nn.Conv2d(ngf * mult * 2, ngf * mult, kernel_size=1, stride=1, bias=True)
        self.relu = nn.ReLU(True)

        # Gamma, Beta block
        if self.light:
            FC = [nn.Linear(ngf * mult, ngf * mult, bias=False),
                  nn.ReLU(True),
                  nn.Linear(ngf * mult, ngf * mult, bias=False),
                  nn.ReLU(True)]
        else:
            FC = [nn.Linear(img_size // mult * img_size // mult * ngf * mult, ngf * mult, bias=False),
                  nn.ReLU(True),
                  nn.Linear(ngf * mult, ngf * mult, bias=False),
                  nn.ReLU(True)]
        self.gamma = nn.Linear(ngf * mult, ngf * mult, bias=False)
        self.beta = nn.Linear(ngf * mult, ngf * mult, bias=False)

        # Up-Sampling Bottleneck
        for i in range(n_blocks):
            setattr(self, 'UpBlock1_' + str(i+1), ResnetAdaILNBlock(ngf * mult, use_bias=False))

        # Up-Sampling
        UpBlock2 = []
        for i in range(n_downsampling):
            mult = 2**(n_downsampling - i)
            UpBlock2 += [nn.Upsample(scale_factor=2, mode='nearest'),
                         nn.ReflectionPad2d(1),
                         nn.Conv2d(ngf * mult, int(ngf * mult / 2), kernel_size=3, stride=1, padding=0, bias=False),
                         ILN(int(ngf * mult / 2)),
                         nn.ReLU(True)]

        UpBlock2 += [nn.ReflectionPad2d(3),
                     nn.Conv2d(ngf, output_nc, kernel_size=7, stride=1, padding=0, bias=False),
                     nn.Tanh()]

        self.DownBlock = nn.Sequential(*DownBlock)
        self.FC = nn.Sequential(*FC)
        self.UpBlock2 = nn.Sequential(*UpBlock2)

    def forward(self, input):
        x = self.DownBlock(input)

        gap = torch.nn.functional.adaptive_avg_pool2d(x, 1)
        gap_logit = self.gap_fc(gap.view(x.shape[0], -1))
        gap_weight = list(self.gap_fc.parameters())[0]
        gap = x * gap_weight.unsqueeze(2).unsqueeze(3)

        gmp = torch.nn.functional.adaptive_max_pool2d(x, 1)
        gmp_logit = self.gmp_fc(gmp.view(x.shape[0], -1))
        gmp_weight = list(self.gmp_fc.parameters())[0]
        gmp = x * gmp_weight.unsqueeze(2).unsqueeze(3)

        cam_logit = torch.cat([gap_logit, gmp_logit], 1)
        x = torch.cat([gap, gmp], 1)
        x = self.relu(self.conv1x1(x))

        heatmap = torch.sum(x, dim=1, keepdim=True)

        if self.light:
            x_ = torch.nn.functional.adaptive_avg_pool2d(x, 1)
            x_ = self.FC(x_.view(x_.shape[0], -1))
        else:
            x_ = self.FC(x.view(x.shape[0], -1))
        gamma, beta = self.gamma(x_), self.beta(x_)


        for i in range(self.n_blocks):
            x = getattr(self, 'UpBlock1_' + str(i+1))(x, gamma, beta)
        out = self.UpBlock2(x)

        return out, cam_logit, heatmap


class ResnetBlock(nn.Module):
    def __init__(self, dim, use_bias):
        super(ResnetBlock, self).__init__()
        conv_block = []
        conv_block += [nn.ReflectionPad2d(1),
                       nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=0, bias=use_bias),
                       nn.InstanceNorm2d(dim),
                       nn.ReLU(True)]

        conv_block += [nn.ReflectionPad2d(1),
                       nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=0, bias=use_bias),
                       nn.InstanceNorm2d(dim)]

        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        out = x + self.conv_block(x)
        return out


class ResnetAdaILNBlock(nn.Module):
    def __init__(self, dim, use_bias):
        super(ResnetAdaILNBlock, self).__init__()
        self.pad1 = nn.ReflectionPad2d(1)
        self.conv1 = nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=0, bias=use_bias)
        self.norm1 = adaILN(dim)
        self.relu1 = nn.ReLU(True)

        self.pad2 = nn.ReflectionPad2d(1)
        self.conv2 = nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=0, bias=use_bias)
        self.norm2 = adaILN(dim)

    def forward(self, x, gamma, beta):
        out = self.pad1(x)
        out = self.conv1(out)
        out = self.norm1(out, gamma, beta)
        out = self.relu1(out)
        out = self.pad2(out)
        out = self.conv2(out)
        out = self.norm2(out, gamma, beta)

        return out + x


class adaILN(nn.Module):
    def __init__(self, num_features, eps=1e-5):
        super(adaILN, self).__init__()
        self.eps = eps
        self.rho = nn.Parameter(torch.Tensor(1, num_features, 1, 1))
        self.rho.data.fill_(0.9)

    def forward(self, input, gamma, beta):
        in_mean, in_var = torch.mean(input, dim=[2, 3], keepdim=True), torch.var(input, dim=[2, 3], keepdim=True)
        out_in = (input - in_mean) / torch.sqrt(in_var + self.eps)
        ln_mean, ln_var = torch.mean(input, dim=[1, 2, 3], keepdim=True), torch.var(input, dim=[1, 2, 3], keepdim=True)
        out_ln = (input - ln_mean) / torch.sqrt(ln_var + self.eps)
        out = self.rho.expand(input.shape[0], -1, -1, -1) * out_in + (1-self.rho.expand(input.shape[0], -1, -1, -1)) * out_ln
        out = out * gamma.unsqueeze(2).unsqueeze(3) + beta.unsqueeze(2).unsqueeze(3)

        return out


class ILN(nn.Module):
    def __init__(self, num_features, eps=1e-5):
        super(ILN, self).__init__()
        self.eps = eps
        self.rho = nn.Parameter(torch.Tensor(1, num_features, 1, 1))
        self.gamma = nn.Parameter(torch.Tensor(1, num_features, 1, 1))
        self.beta = nn.Parameter(torch.Tensor(1, num_features, 1, 1))
        self.rho.data.fill_(0.0)
        self.gamma.data.fill_(1.0)
        self.beta.data.fill_(0.0)

    def forward(self, input):
        in_mean, in_var = torch.mean(input, dim=[2, 3], keepdim=True), torch.var(input, dim=[2, 3], keepdim=True)
        out_in = (input - in_mean) / torch.sqrt(in_var + self.eps)
        ln_mean, ln_var = torch.mean(input, dim=[1, 2, 3], keepdim=True), torch.var(input, dim=[1, 2, 3], keepdim=True)
        out_ln = (input - ln_mean) / torch.sqrt(ln_var + self.eps)
        out = self.rho.expand(input.shape[0], -1, -1, -1) * out_in + (1-self.rho.expand(input.shape[0], -1, -1, -1)) * out_ln
        out = out * self.gamma.expand(input.shape[0], -1, -1, -1) + self.beta.expand(input.shape[0], -1, -1, -1)

        return out


class Discriminator(nn.Module):
    def __init__(self, input_nc, ndf=64, n_layers=5):
        super(Discriminator, self).__init__()
        model = [nn.ReflectionPad2d(1),
                 nn.utils.spectral_norm(
                 nn.Conv2d(input_nc, ndf, kernel_size=4, stride=2, padding=0, bias=True)),
                 nn.LeakyReLU(0.2, True)]

        for i in range(1, n_layers - 2):
            mult = 2 ** (i - 1)
            model += [nn.ReflectionPad2d(1),
                      nn.utils.spectral_norm(
                      nn.Conv2d(ndf * mult, ndf * mult * 2, kernel_size=4, stride=2, padding=0, bias=True)),
                      nn.LeakyReLU(0.2, True)]

        mult = 2 ** (n_layers - 2 - 1)
        model += [nn.ReflectionPad2d(1),
                  nn.utils.spectral_norm(
                  nn.Conv2d(ndf * mult, ndf * mult * 2, kernel_size=4, stride=1, padding=0, bias=True)),
                  nn.LeakyReLU(0.2, True)]

        # Class Activation Map
        mult = 2 ** (n_layers - 2)
        self.gap_fc = nn.utils.spectral_norm(nn.Linear(ndf * mult, 1, bias=False))
        self.gmp_fc = nn.utils.spectral_norm(nn.Linear(ndf * mult, 1, bias=False))
        self.conv1x1 = nn.Conv2d(ndf * mult * 2, ndf * mult, kernel_size=1, stride=1, bias=True)
        self.leaky_relu = nn.LeakyReLU(0.2, True)

        self.pad = nn.ReflectionPad2d(1)
        self.conv = nn.utils.spectral_norm(
            nn.Conv2d(ndf * mult, 1, kernel_size=4, stride=1, padding=0, bias=False))

        self.model = nn.Sequential(*model)

    def forward(self, input):
        x = self.model(input)

        gap = torch.nn.functional.adaptive_avg_pool2d(x, 1)
        gap_logit = self.gap_fc(gap.view(x.shape[0], -1))
        gap_weight = list(self.gap_fc.parameters())[0]
        gap = x * gap_weight.unsqueeze(2).unsqueeze(3)

        gmp = torch.nn.functional.adaptive_max_pool2d(x, 1)
        gmp_logit = self.gmp_fc(gmp.view(x.shape[0], -1))
        gmp_weight = list(self.gmp_fc.parameters())[0]
        gmp = x * gmp_weight.unsqueeze(2).unsqueeze(3)

        cam_logit = torch.cat([gap_logit, gmp_logit], 1)
        x = torch.cat([gap, gmp], 1)
        x = self.leaky_relu(self.conv1x1(x))

        heatmap = torch.sum(x, dim=1, keepdim=True)

        x = self.pad(x)
        out = self.conv(x)

        return out, cam_logit, heatmap


class RhoClipper(object):

    def __init__(self, min, max):
        self.clip_min = min
        self.clip_max = max
        assert min < max

    def __call__(self, module):

        if hasattr(module, 'rho'):
            w = module.rho.data
            w = w.clamp(self.clip_min, self.clip_max)
            module.rho.data = w

In [28]:
class UGATIT(object) :
    def __init__(self):
        self.ch = 64
        self.lr=0.0001
        self.weight_decay=0.0001
        """ Weight """
        self.cycle_weight = 10
        self.identity_weight = 10
        self.cam_weight = 1000

        """ Generator """
        self.n_res = 4

        """ Discriminator """
        self.n_dis = 6

        self.img_size = 128
        self.img_ch = 3

    def build_model(self):

        """ Define Generator, Discriminator """
        genA2B = ResnetGenerator(input_nc=3, output_nc=3, ngf=self.ch, n_blocks=self.n_res, img_size=self.img_size, light=False)
        genB2A = ResnetGenerator(input_nc=3, output_nc=3, ngf=self.ch, n_blocks=self.n_res, img_size=self.img_size, light=False)
        disGA = Discriminator(input_nc=3, ndf=self.ch, n_layers=7)
        disGB = Discriminator(input_nc=3, ndf=self.ch, n_layers=7)
        disLA = Discriminator(input_nc=3, ndf=self.ch, n_layers=5)
        disLB = Discriminator(input_nc=3, ndf=self.ch, n_layers=5)
        self.models=(genA2B,genB2A,disGA,disGB,disLA,disLB)
        

        """ Define Loss """
        L1_loss = nn.L1Loss()
        MSE_loss = nn.MSELoss()
        BCE_loss = nn.BCEWithLogitsLoss()
        self.losses=(L1_loss,MSE_loss,BCE_loss)

        """ Trainer """
        G_optim = torch.optim.Adam(itertools.chain(genA2B.parameters(), genB2A.parameters()), lr=self.lr, betas=(0.5, 0.999), weight_decay=self.weight_decay)
        D_optim = torch.optim.Adam(itertools.chain(disGA.parameters(), disGB.parameters(), disLA.parameters(), disLB.parameters()), lr=self.lr, betas=(0.5, 0.999), weight_decay=self.weight_decay)
        self.optims=(G_optim,D_optim)
        
        """ Define Rho clipper to constraint the value of rho in AdaILN and ILN"""
        self.Rho_clipper = RhoClipper(0, 1)

In [29]:
ugatit=UGATIT()
ugatit.build_model()

In [30]:
noop_loss = lambda x: torch.FloatTensor([0])

In [31]:
class UGATITLoss():
    def __init__(self, models,cycle_weight = 10,identity_weight = 10,cam_weight = 1000,adv_weight=1):
        store_attr(self, "cycle_weight,identity_weight,cam_weight,adv_weight")
        self.model_GA2B,self.model_GB2A,self.model_DA,self.model_DB,self.model_LA,self.model_LB=models
        self.MSE_loss=nn.MSELoss()
        self.L1_loss=nn.L1Loss()
        self.BCE_loss=nn.BCEWithLogitsLoss()
        
    #xb should be pred, need to figure out what to do here
    def __call__(self,xb,yb):
        return self.generator_loss(xb)
    
    def ad_loss(self,dis,fake):
        fake_logit, fake_cam_logit, _ = dis(fake)

        ad_loss = self.MSE_loss(fake_logit, torch.ones_like(fake_logit))
        ad_cam_loss = self.MSE_loss(fake_cam_logit, torch.ones_like(fake_cam_logit))

        return  ad_loss + ad_cam_loss
    
    #fakeB - output of a generator that goes from A2B "B" 
    #realA - real image of type "A"
    #genB2A - generator for going back to A(as in cycle)
    def recon_loss(self,fakeB,realA,genB2A):
        return self.L1_loss(genB2A(fakeB)[0],realA)
            
    def cam_loss(self,fake_A2B_cam_logit,fake_B2B_cam_logit):
        return self.BCE_loss(fake_A2B_cam_logit, torch.ones_like(fake_A2B_cam_logit)) \
            + self.BCE_loss(fake_B2B_cam_logit, torch.zeros_like(fake_B2B_cam_logit))
    def generator_loss(self,xb):
        real_A,real_B=xb
        fake_A2B, fake_A2B_cam_logit, _ = self.model_GA2B(real_A)
        fake_B2A, fake_B2A_cam_logit, _ = self.model_GB2A(real_B)

        fake_A2A, fake_A2A_cam_logit, _ = self.model_GB2A(real_A)
        fake_B2B, fake_B2B_cam_logit, _ = self.model_GA2B(real_B)

        ad_loss_A = self.ad_loss(self.model_DA,fake_B2A) + self.ad_loss(self.model_LA,fake_B2A)
        ad_loss_B = self.ad_loss(self.model_DB,fake_A2B) + self.ad_loss(self.model_LB,fake_A2B)
        loss = ad_loss_A + ad_loss_B

        recon_loss_A = self.recon_loss(fake_A2B,real_A,self.model_GB2A)
        recon_loss_B = self.recon_loss(fake_B2A,real_B,self.model_GA2B)
        loss += self.cycle_weight * (recon_loss_A + recon_loss_B)

        identity_loss_A = self.L1_loss(fake_A2A, real_A)
        identity_loss_B = self.L1_loss(fake_B2B, real_B)
        loss += self.identity_weight * (identity_loss_A + identity_loss_B)

        cam_loss_A = self.cam_loss(fake_B2A_cam_logit,fake_A2A_cam_logit)
        cam_loss_B = self.cam_loss(fake_A2B_cam_logit,fake_B2B_cam_logit)
        return loss + self.cam_weight * (cam_loss_A + cam_loss_B)
    
    def discriminator_loss(self,xb):

        real_A,real_B=xb
        fake_A2B, _, _ = self.model_GA2B(real_A)
        fake_B2A, _, _ = self.model_GB2A(real_B)

        real_GA_logit, real_GA_cam_logit, _ = self.model_DA(real_A)
        real_LA_logit, real_LA_cam_logit, _ = self.model_LA(real_A)
        real_GB_logit, real_GB_cam_logit, _ = self.model_DB(real_B)
        real_LB_logit, real_LB_cam_logit, _ = self.model_LB(real_B)

        fake_GA_logit, fake_GA_cam_logit, _ = self.model_DA(fake_B2A)
        fake_LA_logit, fake_LA_cam_logit, _ = self.model_LA(fake_B2A)
        fake_GB_logit, fake_GB_cam_logit, _ = self.model_DB(fake_A2B)
        fake_LB_logit, fake_LB_cam_logit, _ = self.model_LB(fake_A2B)

        D_ad_loss_GA = self.MSE_loss(real_GA_logit, torch.ones_like(real_GA_logit)) + self.MSE_loss(fake_GA_logit, torch.zeros_like(fake_GA_logit))
        D_ad_cam_loss_GA = self.MSE_loss(real_GA_cam_logit, torch.ones_like(real_GA_cam_logit)) + self.MSE_loss(fake_GA_cam_logit, torch.zeros_like(fake_GA_cam_logit))
        D_ad_loss_LA = self.MSE_loss(real_LA_logit, torch.ones_like(real_LA_logit)) + self.MSE_loss(fake_LA_logit, torch.zeros_like(fake_LA_logit))
        D_ad_cam_loss_LA = self.MSE_loss(real_LA_cam_logit, torch.ones_like(real_LA_cam_logit)) + self.MSE_loss(fake_LA_cam_logit, torch.zeros_like(fake_LA_cam_logit))
        D_ad_loss_GB = self.MSE_loss(real_GB_logit, torch.ones_like(real_GB_logit)) + self.MSE_loss(fake_GB_logit, torch.zeros_like(fake_GB_logit))
        D_ad_cam_loss_GB = self.MSE_loss(real_GB_cam_logit, torch.ones_like(real_GB_cam_logit)) + self.MSE_loss(fake_GB_cam_logit, torch.zeros_like(fake_GB_cam_logit))
        D_ad_loss_LB = self.MSE_loss(real_LB_logit, torch.ones_like(real_LB_logit)) + self.MSE_loss(fake_LB_logit, torch.zeros_like(fake_LB_logit))
        D_ad_cam_loss_LB = self.MSE_loss(real_LB_cam_logit, torch.ones_like(real_LB_cam_logit)) + self.MSE_loss(fake_LB_cam_logit, torch.zeros_like(fake_LB_cam_logit))

        D_loss_A = self.adv_weight * (D_ad_loss_GA + D_ad_cam_loss_GA + D_ad_loss_LA + D_ad_cam_loss_LA)
        D_loss_B = self.adv_weight * (D_ad_loss_GB + D_ad_cam_loss_GB + D_ad_loss_LB + D_ad_cam_loss_LB)

        Discriminator_loss = D_loss_A + D_loss_B
        return Discriminator_loss

In [32]:
class UGATITLearner(Learner):
    "Group together a `model`, some `dbunch` and a `loss_func` to handle training"
    def __init__(self, dbunch, models, loss_funcs=None, opt_funcs=[SGD,SGD], lr=None, splitter=trainable_params, cbs=None,
                 cb_funcs=None, metrics=None, path=None, model_dir='models', wd_bn_bias=False, train_bn=True):
        store_attr(self, "dbunch,splitter,model_dir,wd_bn_bias,train_bn")
        self.model_GA2B,self.model_GB2A,self.model_DA,self.model_DB,self.model_LA,self.model_LB=models
        self.generators=nn.ModuleList([self.model_GA2B,self.model_GB2A])
        self.discriminators=nn.ModuleList([self.model_DA,self.model_DB,self.model_LA,self.model_LB])
        self.model=nn.ModuleList([self.generators,self.discriminators])
        self.models=self.model
        self.opt_funcG,self.opt_funcD=opt_funcs
        self.lr = defaults.lr if lr is None else lr
        #TODO: infer loss_func from data
        self.loss_funcs = L([CrossEntropyLossFlat()]) if loss_funcs is None else loss_funcs
        self.path = path if path is not None else getattr(dbunch, 'path', Path('.'))
        self.metrics = [m if isinstance(m, Metric) else AvgMetric(m) for m in L(metrics)]
        self.training,self.logger,self.opts = False,print,None
        self.cbs = L([])
        self.add_cbs(cbf() for cbf in L(defaults.callbacks[0])) #TODO Took out tracking due ot error
        self.add_cbs(cbs)
        self.add_cbs(cbf() for cbf in L(cb_funcs))
        self.ugatit_loss=UGATITLoss(models)
    
    def create_opt(self,opt_func,models):
        "Create an optimizer with `lr`"
        opt = opt_func(self.splitter(models), lr=self.lr)
        if not self.wd_bn_bias:
            for p in bn_bias_params(models):
                opt.state[p] = {**opt.state.get(p, {}), 'do_wd': False}
        if self.train_bn:
            for p in bn_bias_params(models, with_bias=False):
                opt.state[p] = {**opt.state.get(p, {}), 'force_train': True}
        return opt
    
    def create_opts(self):
        #generator
        optG=self.create_opt(self.opt_funcG,self.generators)
        #discriminator
        optD=self.create_opt(self.opt_funcD,self.discriminators)
        return (optG,optD)

    # models? which models get which inputs?
    def one_batch(self, i, b):
        self.iter = i
        try:
            
            self._split(b)
            print_loss=i%100==0
            if(print_loss):    
                print('batch: #'+str(self.iter)+'/'+str(self.n_iter)+'/'+str(self.epoch))
            if self.epoch*self.n_iter+i > ((self.n_epoch*self.n_iter) // 2):
                self.lr -= (self.lr / ((self.n_epoch*self.n_iter) // 2))
                self.opts[0].set_hyper('lr',self.lr)
                self.opts[1].set_hyper('lr',self.lr)
            self('begin_batch')
            #self.pred = self.model_GA2B(self.xb[0][0]);             self('after_pred')
            l=self.ugatit_loss.generator_loss(self.xb[0])
            self.loss=l.item();         self('after_loss') #should use loss_func
            if not self.training: return
            l.backward();                           self('after_backward')
            if(print_loss): print('G_loss: '+str(l.item()))
            self.opts[0].step();                                self('after_step')
            self.opts[0].zero_grad()
            
            l = self.ugatit_loss.discriminator_loss(self.xb[0]); #self('after_loss2')
            l.backward();                           #self('after_backward2')
            if(print_loss): print('D_loss: '+str(l.item()))
            self.opts[1].step();                                #self('after_step2')
            self.opts[1].zero_grad()
            self.model_GA2B.apply(ugatit.Rho_clipper)
            self.model_GB2A.apply(ugatit.Rho_clipper)
            
        except CancelBatchException:                        self('after_cancel_batch')
        finally:                                            self('after_batch')

    def fit(self, n_epoch, lr=None, wd=None, cbs=None, reset_opt=False):
        "Fit `self.model` for `n_epoch` using `cbs`. Optionally `reset_opt`."
        with self.added_cbs(cbs):
            if reset_opt or not self.opts: self.opts = self.create_opts()
            try:
                self._do_begin_fit(n_epoch)
                for epoch in range(n_epoch):
                    try:
                        self.epoch=epoch;          self('begin_epoch')
                        self._do_epoch_train()
                        self._do_epoch_validate()
                    except CancelEpochException:   self('after_cancel_epoch')
                    finally:                       self('after_epoch')

            except CancelFitException: self('after_cancel_fit')
            finally:                   self('after_fit')


In [33]:
def multi_model_params(models):
    ps=[]
    for m in models:
        ps+=[p for p in m.parameters() if p.requires_grad]
    return ps
learner=UGATITLearner(db,ugatit.models,[noop_loss,noop_loss],[SGD,SGD],0.0001,splitter=multi_model_params)
#.to_fp16() #should use [Adam,Adam]

In [35]:
learner.fit(1000)

batch: #0/23418/0
G_loss: 424017.75
D_loss: 7.879861354827881
batch: #100/23418/0
G_loss: 19244.8203125
D_loss: 5.028888702392578
batch: #200/23418/0
G_loss: 27532.521484375
D_loss: 4.634385585784912
batch: #300/23418/0
G_loss: 7119.2236328125
D_loss: 3.9997103214263916
batch: #400/23418/0
G_loss: 2443.57958984375
D_loss: 4.128623008728027
batch: #500/23418/0
G_loss: 2849.263671875
D_loss: 4.208691596984863
batch: #600/23418/0
G_loss: 16134.548828125
D_loss: 4.085468292236328


KeyboardInterrupt: 

In [ ]:
samples=[]
for i,(xb,yb) in enumerate(iter(db.train_dl)):
    xbA,xbB=xb
    fake_Bb=learner.model_GA2B(xbA)[0].detach()
    samples+=[fake_Bb[0],fake_Bb[1]]
    if(i>10): break

In [ ]:
for i in range(len(samples)):
    show_image(samples[i])

In [ ]:
MixedPrecision??

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

In [ ]:
SaveModelCallback??

In [ ]:
use_partial_data data source use partial data (keep pct) 

In [ ]:
learner.models